In [1]:
%pylab
from numba import jit, njit

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [6]:
class TreeNode:
    def __init__(self, center, size, dim=3):
        self.COM = None
        self.center = center
        self.size = size
        self.mass = None
        self.IsLeaf = False
        self.children = (1 << dim) * [None,]
        self.dim = dim
        
    def InsertPoint(self, x, m):
        if self.COM is None:
            self.COM  = x
            self.mass = m
            self.IsLeaf = True
            return
        #otherwise we gotta split this up
        if self.IsLeaf:
            self.SpawnChildWithPoint(self.COM, self.mass)
            self.IsLeaf = False    
        self.SpawnChildWithPoint(x, m)
    
    def SpawnChildWithPoint(self, x, m):
        signs = (x > self.center)
        sector = sum(signs[i] * (1 << i) for i in range(self.dim))
        if not self.children[sector]: 
            self.children[sector] = TreeNode(self.center + 0.5*(signs-0.5) * self.size, self.size/2, dim=self.dim)
        self.children[sector].InsertPoint(x, m)
        
    def GetMoments(self):
        if not self.IsLeaf: #: return self.mass, self.COM
            self.mass = 0.
            self.COM = np.zeros(self.dim)
            for c in self.children:
                if c is None: continue
                mc, xc = c.GetMoments()
                self.mass += mc
                self.COM += mc*xc
            self.COM /= self.mass
        return self.mass, self.COM

    
def ForceWalk(x, g, node, thetamax=0.7, eps=0.0):
    dx = node.COM - x
    #print(dx)
    r = np.sqrt((dx**2).sum())
    if r>0:
        if node.IsLeaf or node.size/r < thetamax:
            g += node.mass * dx / (r**2 + eps**2)**1.5
        else:
            for c in node.children:
                if c: ForceWalk(x, g, c, thetamax, eps)

def Accel(points, tree, thetamax=0.7, G=1.0, eps=0.0):
    accels = np.zeros_like(points)
    for i in range(points.shape[0]):
        ForceWalk(points[i], accels[i], tree, thetamax,eps)
    return G*accels

@njit
def BruteForceAccel(x,m,eps=0., G=1.):
    accel = zeros_like(x)
    for i in range(x.shape[0]):
        for j in range(i+1,x.shape[0]):
            dx = x[j,0]-x[i,0]
            dy = x[j,1]-x[i,1]
            dz = x[j,2]-x[i,2]
            r = sqrt(dx*dx + dy*dy + dz*dz + eps*eps)
            mr3inv = m[i]/(r*r*r)
            accel[j,0] -= mr3inv*dx
            accel[j,1] -= mr3inv*dy
            accel[j,2] -= mr3inv*dz

            mr3inv = m[j]/(r*r*r)
            accel[i,0] += mr3inv*dx
            accel[i,1] += mr3inv*dy
            accel[i,2] += mr3inv*dz
    return G*accel

@jit
def BruteForcePotential(x,m,G=1., eps=0.):
    potential = np.zeros_like(m)
    for i in range(x.shape[0]):
        for j in range(i+1,x.shape[0]):
            dx = x[i,0]-x[j,0]
            dy = x[i,1]-x[j,1]
            dz = x[i,2]-x[j,2]
            r = np.sqrt(dx*dx + dy*dy + dz*dz + eps*eps)
            rinv = 1/r
            potential[j] -= m[i]*rinv
            potential[i] -= m[j]*rinv
    return G*potential

def ConstructTree(points, masses):
    mins = np.min(points,axis=0)
    maxes = np.max(points,axis=0)
    center = (maxes+mins)/2
    size = np.max(maxes-mins)
    root = TreeNode(center, size, dim=points.shape[1])
    for i in range(len(points)):
        root.InsertPoint(points[i], masses[i])
    root.GetMoments()
    return root


In [14]:
x = 2*(np.random.rand(200,3) - 0.5)
x = x[np.sum(x**2,axis=1)<1.]
#x[:,2] /= 10
masses = np.repeat(1/x.shape[0],x.shape[0])
v = np.cross(x, np.array([0,0,1])) * 3
v += np.random.normal(size=x.shape)*0.1
v *= 0.
#plt.scatter(x[:,0], x[:,1]); plt.show()

In [18]:
#g = np.zeros(3)
eps = 0.1
root = ConstructTree(x, masses)
a = Accel(x, root, thetamax=0.7,eps=0.1)
a[0], BruteForceAccel(x,masses,eps)[0]
#plt.hist(a[:,1],100); plt.show()
#root.children#[2].center
#x[np.sum(a**2,axis=1).argmax()]
#BruteForceAccel(points, masses)

(array([-0.44515349, -0.56516813,  2.33472878]),
 array([-0.45022776, -0.57476178,  2.28688456]))

In [15]:
dt = 0.001
eps = 0.1
t = 0.
tmax = 1.
i = 0
#plt.ion()
#ion()
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.axes().set_aspect('equal'); 
#plt.xlim(-1,1)
#plt.ylim(-1,1)
KE = []
PE = []
med = []
while t < tmax:
    if not i%100: print(t)# ax.clear(); ax.scatter(x[:,0],x[:,1],s=0.3); plt.xlim(-1,1); plt.ylim(-1,1); plt.draw(); plt.pause(0.01)
    #plt.savefig("%d.png"%i); plt.plt.clf()
    x += v*dt #, v + BruteForceAccel(x, masses, eps=eps)*dt
    #root = ConstructTree(x, masses)
    v += BruteForceAccel(x, masses, eps=eps)*dt
    i += 1
    t += dt
    KE.append((v**2).sum())
    PE.append(BruteForcePotential(x,masses,1.,eps).sum())
    med.append(np.percentile(np.sum(x**2,axis=1)**0.5, 50))

0.0
0.10000000000000007
0.20000000000000015
0.3000000000000002
0.4000000000000003
0.5000000000000003
0.6000000000000004
0.7000000000000005
0.8000000000000006
0.9000000000000007


In [17]:
plt.plot(np.array(KE) + np.array(PE))
#plt.plot(PE)
plt.show()

In [86]:
plt.plot(med); plt.show()

In [ ]:
np.packbits(np.array([True,True,True]))

In [ ]:
1 >> 0

In [ ]:
[1 << i for i in range(2)]